In [12]:
# !pip install pydicom
# !pip install dicomweb_client
# !pip install tqdm

In [1]:
from dicomweb_client.api import DICOMwebClient
import pydicom
import os
import time
import datetime
from tqdm import tqdm
import csv
import random

url = 'http://localhost:8080/dcm4chee-arc/aets/DCM4CHEE/rs'

client = DICOMwebClient(url)

Funkcja obliczania długości odczytywania plików z bazy:

In [23]:
sample_size = 100
def get_average_read_time(patient_UID):
    # warmup reads (for the first time search time is larger for some reason - idk why)
    client.search_for_studies(search_filters={'PatientID': str(patient_UID)})
    client.search_for_studies(search_filters={'PatientID': str(patient_UID)})
    
    samples = []
    for i in range(sample_size):
        samples.append(random.randint(0, patient_UID))
    
    start_time = time.time()
    
    for sample in samples:
        print(sample)
        client.search_for_studies(search_filters={'PatientID': str(sample)})
    
    search_time = (time.time() - start_time) / sample_size
    
    print("Patient UDI: " + str(patient_UID) + "; average search time: " + str(search_time) )
    return search_time

get_average_read_time(100000)

empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


50269
83960
21290
5245
98955
44588
90112
71891
99164
95629
92002
22587
57101
21617
99104
87681
95971
43617
82354
9256


empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


63010
97264
860
62956
61480
64026
75949
97318
95106
11690
53864
85448
78559
15014
10243
57278
92882
15967
59739
68897
22301
83108


empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


...

empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


27998
22737
1589
81282
50617
57235
42581
21822
50026
966
64152
34930
88322
97897
8991
2067
41291
76608
88528
37322


empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


59103
2556
85891
51565
243
82171
55435
22881
94270
11565
65966
28320
89371
43860
66677
Patient UDI: 100000; average search time: 0.008646788597106934


0.008646788597106934

Przygotowanie pliku z logami:

In [3]:
path = "log/uploading_dicom_to_pacs_logs-" + str(time.strftime("%Y%m%d-%H%M%S")) + ".csv"
print("logs path: " + path)
header = ['timestamp', 'number_of_images', 'last_1k_dicoms_uploadtime', 'experiment_time', 'pacs_read_time']

with open(path, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

logs path: log/uploading_dicom_to_pacs_logs-20220524-225458.csv


Funkcja wgrywania paczki dicomów do pacsa:

In [4]:
def upload_1k_dicoms_into_pacs(images_prefix):
    start_time = time.time()
    
    for i in range(1000):
        uid = str(images_prefix) + str(i)
        
        dicom.SeriesInstanceUID = uid
        dicom.StudyInstanceUID = uid
        dicom.FrameOfReferenceUID = uid
        dicom.SOPInstanceUID = uid
        dicom.PatientID = uid
        client.store_instances(datasets=[dicom])
    
    return( time.time() - start_time )   

### Zapełnianie bazy dicomami:

Przygotowanie dicoma:

In [5]:
dicom = pydicom.dcmread('test_images/IM-0014-0001.dcm')
dicom.PixelData = bytes()

Pętla wgrywania:

In [ ]:
uid_prefix = dicom.SeriesInstanceUID + '.'

experiment_start_time = time.time()

for i in tqdm(range(250,1000)):
    upload_time = upload_1k_dicoms_into_pacs(uid_prefix + str(i))
    
#     timestamp = datetime.datetime.now()
#     number_of_images = i*1000
#     last_1k_dicoms_uploadtime = upload_time
#     experiment_time = time.time() - experiment_start_time
#     pacs_read_time = get_average_read_time(number_of_images)
    
#     data = [str(timestamp), str(number_of_images), str(last_1k_dicoms_uploadtime), str(experiment_time), str(pacs_read_time)]
    
#     with open(path, 'a', encoding='UTF8', newline='') as f:
#         csv.writer(f).writerow(data)
    

 84%|███████████████████████████████████████████████████████████████▋            | 629/750 [9:11:46<1:46:21, 52.74s/it]